In [5]:
val spark = SparkSession.builder.appName("Simple Application").getOrCreate()
// For implicit conversions from RDDs to DataFrames
import spark.implicits._
import org.apache.spark.sql.types._
import org.apache.spark.sql._

spark = org.apache.spark.sql.SparkSession@45b792ee


org.apache.spark.sql.SparkSession@45b792ee

In [6]:
val spark_home=sys.env("SPARK_HOME")

spark_home = /Users/xiaofeiwu/jcloud/assets/langs/workspace/spark/spark-2.4.0-bin-hadoop2.7


/Users/xiaofeiwu/jcloud/assets/langs/workspace/spark/spark-2.4.0-bin-hadoop2.7

In [9]:
case class Rating(userId: Int, movieId: Int, rating: Float, timestamp: Long)

defined class Rating


In [10]:
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.recommendation.ALS

def parseRating(str: String): Rating = {
  val fields = str.split("::")
  assert(fields.size == 4)
  Rating(fields(0).toInt, fields(1).toInt, fields(2).toFloat, fields(3).toLong)
}

val ratings = spark.read.textFile(spark_home+"/data/mllib/als/sample_movielens_ratings.txt")
  .map(parseRating)
  .toDF()
val Array(training, test) = ratings.randomSplit(Array(0.8, 0.2))

// Build the recommendation model using ALS on the training data
val als = new ALS()
  .setMaxIter(5)
  .setRegParam(0.01)
  .setUserCol("userId")
  .setItemCol("movieId")
  .setRatingCol("rating")
val model = als.fit(training)

// Evaluate the model by computing the RMSE on the test data
// Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
model.setColdStartStrategy("drop")
val predictions = model.transform(test)

val evaluator = new RegressionEvaluator()
  .setMetricName("rmse")
  .setLabelCol("rating")
  .setPredictionCol("prediction")
val rmse = evaluator.evaluate(predictions)
println(s"Root-mean-square error = $rmse")

// Generate top 10 movie recommendations for each user
val userRecs = model.recommendForAllUsers(10)
// Generate top 10 user recommendations for each movie
val movieRecs = model.recommendForAllItems(10)

// Generate top 10 movie recommendations for a specified set of users
val users = ratings.select(als.getUserCol).distinct().limit(3)
val userSubsetRecs = model.recommendForUserSubset(users, 10)
// Generate top 10 user recommendations for a specified set of movies
val movies = ratings.select(als.getItemCol).distinct().limit(3)
val movieSubSetRecs = model.recommendForItemSubset(movies, 10)

Root-mean-square error = 1.8438530830091973


ratings = [userId: int, movieId: int ... 2 more fields]
training = [userId: int, movieId: int ... 2 more fields]
test = [userId: int, movieId: int ... 2 more fields]
als = als_3329f7cddff7
model = als_3329f7cddff7
predictions = [userId: int, movieId: int ... 3 more fields]
evaluator = regEval_5be120fc90c3
rmse = 1.8438530830091973


parseRating: (str: String)Rating
userRe...


1.8438530830091973

In [18]:
userRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    28|[[65, 5.9662914],...|
|    26|[[46, 7.264442], ...|
|    27|[[46, 3.9809933],...|
|    12|[[46, 5.42597], [...|
|    22|[[52, 5.619651], ...|
|     1|[[68, 3.6758366],...|
|    13|[[25, 3.6470919],...|
|     6|[[25, 4.794844], ...|
|    16|[[39, 5.2071724],...|
|     3|[[51, 4.9299946],...|
|    20|[[22, 4.8744855],...|
|     5|[[55, 4.4801764],...|
|    19|[[90, 3.879931], ...|
|    15|[[46, 4.462626], ...|
|    17|[[20, 5.0643454],...|
|     9|[[49, 5.0883746],...|
|     4|[[52, 4.085467], ...|
|     8|[[29, 4.990096], ...|
|    23|[[25, 6.396133], ...|
|     7|[[64, 4.1876116],...|
+------+--------------------+
only showing top 20 rows



In [19]:
movieRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     31|[[12, 4.050344], ...|
|     85|[[16, 4.830504], ...|
|     65|[[28, 5.9662914],...|
|     53|[[22, 4.3496494],...|
|     78|[[26, 1.377129], ...|
|     34|[[2, 4.089016], [...|
|     81|[[28, 5.1141415],...|
|     28|[[18, 5.043909], ...|
|     76|[[14, 4.9924316],...|
|     26|[[0, 2.7917144], ...|
|     27|[[12, 5.034793], ...|
|     44|[[18, 3.9973888],...|
|     12|[[28, 4.98783], [...|
|     91|[[23, 3.7962852],...|
|     22|[[26, 5.00882], [...|
|     93|[[28, 4.4008946],...|
|     47|[[7, 4.068767], [...|
|      1|[[15, 3.692736], ...|
|     52|[[22, 5.619651], ...|
|     13|[[9, 4.2738132], ...|
+-------+--------------------+
only showing top 20 rows



In [20]:
userSubsetRecs.show()
movieSubSetRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    28|[[65, 5.9662914],...|
|    26|[[46, 7.264442], ...|
|    27|[[46, 3.9809933],...|
+------+--------------------+

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     31|[[12, 4.050344], ...|
|     85|[[16, 4.830504], ...|
|     65|[[28, 5.9662914],...|
+-------+--------------------+



In [22]:
movies.show()
users.show()

+-------+
|movieId|
+-------+
|     31|
|     85|
|     65|
+-------+

+------+
|userId|
+------+
|    28|
|    26|
|    27|
+------+

